In [1]:
# pip install spotipy 

# Imports

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np 
import time 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score

# Credentials

Credentials were stored in another file and I'm going to be importing them.

In [3]:
creds= pd.read_csv('../credentials')
SPOTIPY_CLIENT_ID = creds['Client ID'][0]
SPOTIPY_CLIENT_SECRET = creds['Client Secret'][0]
#Shameless redirect to my dance group lol 
SPOTIPY_REDIRECT_URI= 'https://www.ballethermosoamanecer.com/'
username = ""

# Data Collection 

## Testing 20 Songs in Liked 

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id= SPOTIPY_CLIENT_ID, 
                                                      client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)
scope = 'user-library-read playlist-read-private playlist-modify-private user-read-recently-played user-top-read'
token = util.prompt_for_user_token(username, scope, 
                                   client_id= SPOTIPY_CLIENT_ID, 
                                   client_secret= SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri= SPOTIPY_REDIRECT_URI)
if token:
    
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    count = 0
    total_liked_songs = results['total']
    for item in results['items']:
        track = item['track']
        print(track['name'] + ' - ' + track['artists'][0]['name'])
        count += 1
    print(f"\nShowing {count} out of {total_liked_songs} songs")
else:
    print("Can't get token for", username)

Atrévete - Nicky Jam
Cyanide - Metallica
El Esqueleto - Victimas Del Doctor Cerebro
A Través Del Vaso - Banda Los Sebastianes
Dale Vieja Dale - Toño Rosario
Las Amo a Las Dos - Cumbias Poblanas
Nunca Es Suficiente - Los Angeles Azules
Yo También - Romeo Santos
Gotas De Lluvia - Grupo Niche
La Cura - Frankie Ruiz
Casi Te Envidio - Andy Montañez
Tu Me Quemas - Eddie Santiago
Idilio - Willie Colón
Te Va a Doler - Maelo Ruiz
Corazon de Acero - Yiyo Sarante
Ignorantes - Bad Bunny
Eres Mi Sueño - Septeto Acarey
La Dosis Perfecta - Panteon Rococo
Devuélveme a mi chica - Hombres G
Disco Drum - Black Caviar

Showing 20 out of 995 songs


Clealy my listening habits are... a little diverse haha 

## Collecting Audio Features for Liked Songs

In [5]:
song_name = []
song_uri = []
song_popularity = []
artist_name = []
artist_uri = []
genres_found = []
loops= 0 
track_audio_features = []
while results:
    for item in results['items']:
        track = item['track']
        song_name.append(track['name'])
        song_uri.append(track['uri'])
        song_popularity.append(track['popularity'])
    
        #Some songs have multuple artists so I'll make a list of artists as well as a list of uris for each song
        artist_name.append([artist['name'] for artist in track['artists']])
        artist_uri.append([artist['uri'] for artist in track['artists']])
    
        #Spotify doesn't provide the genre of the song so I have to look at the genre of the artist(s) of the song
        temp = sp.artists([artist['uri'] for artist in track['artists']])['artists']
        temp2 = []
        for temp_artists in temp:
            for genre in temp_artists['genres']:
                temp2.append(genre)
        #We only want to keep unique genres and avoid duplicate genres
        genres_found.append(list(set(temp2)))
    
    #Collecting audio features available for each song 
    #As shown from above, 20 
    track_audio_features.extend(sp.audio_features(song_uri[20*loops :20*(loops+1)]))
    print(f'Audio Features were gathered for {len(track_audio_features)} songs')

    loops += 1 
    results = sp.next(results)
    if (loops % 5) == 0 :
        print(f'{loops} cycles have been completed\n')
    #Sleep in order to not bombard with too many requests
    time.sleep(.8)

#Bringing it all together 
df_audio_feat = pd.DataFrame(track_audio_features) 
df_tracks = pd.DataFrame({'artist_uri': artist_uri,
                          'song_uri': song_uri,
                          'song_name':song_name,
                          'artists': artist_name,
                          'genres': genres_found,
                          'popularity': song_popularity})

print("\nCreating Dataframe . . .")
collected_data = pd.merge(df_tracks, df_audio_feat, left_on= 'song_uri', right_on= 'uri')
                          

Audio Features were gathered for 20 songs
Audio Features were gathered for 40 songs
Audio Features were gathered for 60 songs
Audio Features were gathered for 80 songs
Audio Features were gathered for 100 songs
5 cycles have been completed

Audio Features were gathered for 120 songs
Audio Features were gathered for 140 songs
Audio Features were gathered for 160 songs
Audio Features were gathered for 180 songs
Audio Features were gathered for 200 songs
10 cycles have been completed

Audio Features were gathered for 220 songs
Audio Features were gathered for 240 songs
Audio Features were gathered for 260 songs
Audio Features were gathered for 280 songs
Audio Features were gathered for 300 songs
15 cycles have been completed

Audio Features were gathered for 320 songs
Audio Features were gathered for 340 songs
Audio Features were gathered for 360 songs
Audio Features were gathered for 380 songs
Audio Features were gathered for 400 songs
20 cycles have been completed

Audio Features were g

In [6]:
collected_data.head()

,artist_uri,song_uri,song_name,artists,genres,popularity,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,"[spotify:artist:1SupJlEpv7RS2tPNRaHViT, spotif...",spotify:track:3KekO5WUfJsh7kYU9o2w50,Atrévete,"[Nicky Jam, Sech]","[latin hip hop, latin, tropical, panamanian po...",46,0.552,0.675,11,-3.824,...,0.1190,0.586,183.785,audio_features,3KekO5WUfJsh7kYU9o2w50,spotify:track:3KekO5WUfJsh7kYU9o2w50,https://api.spotify.com/v1/tracks/3KekO5WUfJsh...,https://api.spotify.com/v1/audio-analysis/3Kek...,198320,4
1,[spotify:artist:2ye2Wgw4gimLv2eAKyk1NB],spotify:track:2LfonCFEOzFedikP12rTeo,Cyanide,[Metallica],"[metal, speed metal, rock, hard rock, thrash m...",43,0.313,0.993,11,-0.463,...,0.3030,0.497,129.255,audio_features,2LfonCFEOzFedikP12rTeo,spotify:track:2LfonCFEOzFedikP12rTeo,https://api.spotify.com/v1/tracks/2LfonCFEOzFe...,https://api.spotify.com/v1/audio-analysis/2Lfo...,399547,4
2,[spotify:artist:6Z112eJxKl1E3nAbYZBr7M],spotify:track:1Z05BEFLTKxdp9xSjUnPGX,El Esqueleto,[Victimas Del Doctor Cerebro],"[latin alternative, rock en espanol, mexican r...",57,0.543,0.781,7,-9.249,...,0.5140,0.700,150.395,audio_features,1Z05BEFLTKxdp9xSjUnPGX,spotify:track:1Z05BEFLTKxdp9xSjUnPGX,https://api.spotify.com/v1/tracks/1Z05BEFLTKxd...,https://api.spotify.com/v1/audio-analysis/1Z05...,221640,4
3,[spotify:artist:0HgICyWHmS6rnl8xWEd0x6],spotify:track:1uqjLVaSyTkdDvfGAmXrHo,A Través Del Vaso,[Banda Los Sebastianes],"[regional mexican, banda, regional mexican pop]",66,0.649,0.529,2,-4.895,...,0.0563,0.964,144.869,audio_features,1uqjLVaSyTkdDvfGAmXrHo,spotify:track:1uqjLVaSyTkdDvfGAmXrHo,https://api.spotify.com/v1/tracks/1uqjLVaSyTkd...,https://api.spotify.com/v1/audio-analysis/1uqj...,180120,4
4,[spotify:artist:4DjY0CaKAejJgghVHTaOUy],spotify:track:734vYzX6YKgSDHzXuLQJmg,Dale Vieja Dale,[Toño Rosario],"[tropical, latin, dominican pop, merengue]",53,0.648,0.951,7,-1.565,...,0.0707,0.966,165.364,audio_features,734vYzX6YKgSDHzXuLQJmg,spotify:track:734vYzX6YKgSDHzXuLQJmg,https://api.spotify.com/v1/tracks/734vYzX6YKgS...,https://api.spotify.com/v1/audio-analysis/734v...,216503,4


Alright, now there are two options. 

Ideally, I want to create playlists for each genre and place the correct songs into each playlist. Given that some songs can be classified under multiple genres (since we're using the artist's genere as a proxy), it might be difficult to classify songs into genre's that spotify has provided. In addition, genres are waaaaay too specific for my needs. To start, I will simplify my problem and try classying my music into larger, umbrella genres. I will also explore KMeans clustering in order to see how many realistic clusters can be formed. I will use PCA in order to reduce dimensionality and see if that improves the KMeans. I will repeat the processes with DBSCAN and see if there are any improvements as well as noticable outliers. 


# Data Cleaning

## Outliers and  `null` values

We have some data cleaning to do before I can use my data. Let's check for `null` values and remove columns that are repetitive and provide no additional information. Domain knowledge usually helps identify any outliers or data points that might be off due to human error but for this case, I will assume all values provided by Spotify are normal. 

In [7]:
collected_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 995 entries, 0 to 994
Data columns (total 24 columns):
artist_uri          995 non-null object
song_uri            995 non-null object
song_name           995 non-null object
artists             995 non-null object
genres              995 non-null object
popularity          995 non-null int64
danceability        995 non-null float64
energy              995 non-null float64
key                 995 non-null int64
loudness            995 non-null float64
mode                995 non-null int64
speechiness         995 non-null float64
acousticness        995 non-null float64
instrumentalness    995 non-null float64
liveness            995 non-null float64
valence             995 non-null float64
tempo               995 non-null float64
type                995 non-null object
id                  995 non-null object
uri                 995 non-null object
track_href          995 non-null object
analysis_url        995 non-null object
duration_

In [8]:
collected_data.isnull().sum()

artist_uri          0
song_uri            0
song_name           0
artists             0
genres              0
popularity          0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
type                0
id                  0
uri                 0
track_href          0
analysis_url        0
duration_ms         0
time_signature      0
dtype: int64

## Dropping Unecessary Columns

In [9]:
data= collected_data.drop(columns= ['artist_uri', 'type','id', 'uri', 'track_href', 'analysis_url'])
data

,song_uri,song_name,artists,genres,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:3KekO5WUfJsh7kYU9o2w50,Atrévete,"[Nicky Jam, Sech]","[latin hip hop, latin, tropical, panamanian po...",46,0.552,0.675,11,-3.824,1,0.2970,0.311000,0.000000,0.1190,0.586,183.785,198320,4
1,spotify:track:2LfonCFEOzFedikP12rTeo,Cyanide,[Metallica],"[metal, speed metal, rock, hard rock, thrash m...",43,0.313,0.993,11,-0.463,0,0.0751,0.000119,0.000335,0.3030,0.497,129.255,399547,4
2,spotify:track:1Z05BEFLTKxdp9xSjUnPGX,El Esqueleto,[Victimas Del Doctor Cerebro],"[latin alternative, rock en espanol, mexican r...",57,0.543,0.781,7,-9.249,1,0.1530,0.038000,0.000000,0.5140,0.700,150.395,221640,4
3,spotify:track:1uqjLVaSyTkdDvfGAmXrHo,A Través Del Vaso,[Banda Los Sebastianes],"[regional mexican, banda, regional mexican pop]",66,0.649,0.529,2,-4.895,0,0.0398,0.621000,0.000000,0.0563,0.964,144.869,180120,4
4,spotify:track:734vYzX6YKgSDHzXuLQJmg,Dale Vieja Dale,[Toño Rosario],"[tropical, latin, dominican pop, merengue]",53,0.648,0.951,7,-1.565,1,0.2330,0.527000,0.000000,0.0707,0.966,165.364,216503,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,spotify:track:0HDHY6RSHHG2ZTE0cMT4GJ,Los Infieles,[Aventura],"[latin hip hop, bachata, latin, latin pop, tro...",69,0.745,0.716,9,-8.221,0,0.0411,0.202000,0.000000,0.0590,0.817,132.932,257187,4
991,spotify:track:2XYkvc4UWMO9U2iQcIjJe7,Te Extraño - Bachata Version,[Xtreme],"[tropical, bachata, latin]",2,0.793,0.592,6,-4.749,0,0.0344,0.760000,0.000000,0.0876,0.892,129.960,213973,4
992,spotify:track:0wDEs6WvqDHq4XJZC0dHhO,Hoja En Blanco,"[Monchy & Alexandra, Alexandra]","[tropical, bachata, latin]",63,0.932,0.772,0,-3.599,1,0.0369,0.225000,0.000003,0.1760,0.863,135.063,307200,4
993,spotify:track:6I86RF3odBlcuZA9Vfjzeq,Eres Mía,[Romeo Santos],"[tropical, bachata, latin]",72,0.843,0.729,6,-3.634,0,0.0374,0.401000,0.000001,0.2060,0.903,123.046,250640,4


## Checking for Duplicates 

In [10]:
data[data.duplicated(subset=['song_uri'])]

,song_uri,song_name,artists,genres,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


No duplicates found

## Simplifying Generes (Manually) 

Lets see what genres we have:

In [11]:
genre_dummied = data['genres'].str.join(sep='*').str.get_dummies(sep='*')
genre_dummied

,adult standards,african percussion,afro dancehall,afropop,album rock,alternative dance,alternative metal,alternative r&b,alternative rock,argentine hip hop,...,venezuelan rock,veracruz indie,viral pop,vocal jazz,welsh metal,west coast rap,world worship,worship,wrestling,yacht rock
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
991,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
993,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In theory, I would have 365 different playlists if I made a playlist for each genre. That's excessive and unproductive since some songs pertain to multiple genres and some genres only have one song associated with it. Lets add up our genres and see what's popular in my liked songs 

In [12]:
genre_dummied.sum().sort_values(ascending= False)

latin               332
tropical            223
reggaeton           217
grupera             135
latin pop           133
                   ... 
kids dance party      1
jazz guitar           1
irish hip hop         1
indie folk            1
lounge                1
Length: 367, dtype: int64

Given that `latin` is the most common genere, I will go back to `data` and check if any `latin` is in the list of generes. I will replace the list with `latin` in order to simplfy my process. There are multiple subgeneres like, cumbia, bachata, merenguem etc. but for the initial phase, I am focusing on latin or not latin.

--------
After initial EDA, I have to modify my problem since my songs are overwhelmingly Hispanic. I made subdivisions of the different subgenres within Hispanic music. Not as accurate but this is how I distinguish my own music tastes and this is how I want my playlists to be organized by (for the most part). I notcied that some artists do not have a genre associated with them so lets figure out why and what to do with these values 

In [13]:
bachata_salsa_merengue = {'porro','bachata', 'merengue','salsa', 'salsa peruana', 'tropical', 'timba', 'cuban rumba','dominican pop'}
artists_bsm = []

cumbia_and_mexican = {'trival','cumbia','cumbia salvadorena','gruperas inmortales' ,'cumbia villera', 
                      'cumbia sonidera', 'tejano', 'guaracha', 'cumbia paraguaya','nu-cumbia', 
                      'grupera','tamborazo','banda', 'ranchera', 'mariachi', 'duranguense', 
                      'cancion melodica', 'norteno-sax','regional mexican','regional mexican pop'}
artists_cam = []

latin_reggaeton = {'rap dominicano','perreo','venezuelan hip hop','chilean hardcore','reggaeton', 
                   'dembow', 'trap latino', 'latin hip hop','panamanian pop','venezuelan indie',
                   'latin pop', 'mexican edm', 'pop romantico', 'reggaeton flow', 'colombian pop','electro latino',
                   'puerto rican pop'}
artists_lat_reg = []

not_hispanic = {'escape room','christian music','shimmer pop','bass trap' ,
                'emo rap','indie pop','indie soul', 'edm', 'house', 
                'british soul', 'disco', 'funk','la pop','uk hip hop',
                'social media pop','afropop','reggae fusion','ghanaian pop','soca',
                'hip hop','pop rap', 'pop', 'hip pop', 'rap', 
                'chicago rap','chicago pop', 'meme rap', 'dancehall'}
artists_not_his = []

rock = {'mexican rock', 'rock en espanol', 'latin rock', 'mexican rock-and-roll',
        'metalcore','screamo','rock', 'rock-and-roll', 'punk', 'alternative metal', 
        'soft rock', 'glam rock', 'piano rock'}
artists_rock = []

empty_index = []


In [14]:
simplified_genres = []
for i in range(len(genres_found)):
    counter_a = len(not_hispanic & set(genres_found[i]))
    counter_b = len(rock & set(genres_found[i]))
    counter_c = len(cumbia_and_mexican & set(genres_found[i]))
    counter_d = len(bachata_salsa_merengue  & set(genres_found[i]))
    counter_e = len(latin_reggaeton & set(genres_found[i]))

    count_max = max([counter_a, counter_b, counter_c, counter_d, counter_e])
    
    if count_max == 0:
        simplified_genres.append('Empty')
        empty_index.append(i)
        
    elif count_max == counter_a:    
        simplified_genres.append('Non Hispanic')
        artists_not_his.extend(artist_name[i])
        
    elif count_max == counter_b:
        simplified_genres.append('Rock')
        artists_rock.extend(artist_name[i])

        
    elif count_max == counter_c:
        simplified_genres.append('Cumbia y Musica Mexicana')
        artists_cam.extend(artist_name[i])
       
    elif count_max == counter_d:
        simplified_genres.append('Bachata Salsa y Merengue')
        artists_bsm.extend(artist_name[i])

    elif count_max == counter_e:
        simplified_genres.append('Latin Pop and Regg')
        artists_lat_reg.extend(artist_name[i])



data['simplified_genre'] = simplified_genres

In [15]:
genre_dummied = data['simplified_genre'].str.get_dummies()
genre_dummied

,Bachata Salsa y Merengue,Cumbia y Musica Mexicana,Empty,Latin Pop and Regg,Non Hispanic,Rock
0,0,0,0,1,0,0
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,1,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
990,1,0,0,0,0,0
991,1,0,0,0,0,0
992,1,0,0,0,0,0
993,1,0,0,0,0,0


In [16]:
genre_dummied.sum().sort_values(ascending= False) 

Cumbia y Musica Mexicana    313
Latin Pop and Regg          207
Non Hispanic                203
Rock                        119
Bachata Salsa y Merengue     87
Empty                        66
dtype: int64

Looks like there are some artists that do not have genre data. I'm not sure why that is but since we used a list, it would not have appeared as a null value. Lets check out what `Empty` genre looks like

In [17]:
data[data['simplified_genre'] == 'Empty'][15:20]

,song_uri,song_name,artists,genres,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,simplified_genre
247,spotify:track:6bGYUzH3Cv74Skv4J0QQGR,Corazon Enamorado,[La Conga],[],29,0.645,0.746,9,-2.979,0,0.1850,0.3530,0.000313,0.0651,0.685,176.941,215453,4,Empty
258,spotify:track:1cXHJnw8YvkOygZCCiTbSQ,Mil Horas,[Sonora Dinamica De Colombia],[],0,0.855,0.457,0,-12.351,1,0.0512,0.0129,0.006780,0.0689,0.837,99.983,231187,4,Empty
262,spotify:track:5LNYnwMBaqUkNmTiPbRsoY,Fuiste Mala,"[A.B. Quintanilla III Y Los Kumbia Kings, Rica...",[],51,0.682,0.742,0,-3.383,0,0.0477,0.3940,0.000000,0.1680,0.915,172.019,198213,4,Empty
264,spotify:track:1fATLOTF1CZR7ZM6EVNmUN,El Paso del Gigante,[Sonador],[],55,0.564,0.785,9,-6.470,0,0.1880,0.0924,0.000000,0.0965,0.780,179.885,218107,4,Empty
276,spotify:track:71J3ktUHU5n0EWAfD7VVFX,"Tao, Tao",[Control],[],55,0.854,0.848,7,-5.930,1,0.0410,0.2340,0.000067,0.0294,0.949,103.321,172707,4,Empty


Oh man, it looks like I will have to manually assign them to categories. Let the listening begin. I'll reassign the simplfied genres to make sure we have no more empty. I will also add the artists to their respective artists list in case I need to use it in the future. 


After this, I'll make playlists based on some machine learning models. The bright side is that since I'm only concerned with Hispanic genres, anything with an english title will be placed into non-hispanic. 

In [18]:
artists_bsm.extend(['Kalimete', 'Merenglass Grupo', 'Célia', 'Jeyro', 'Judy Santos', 
                    'Orquesta los Adolecente', 'Orquesta Noche Sabrosa'])
artists_cam.extend(['Grupo Dinastia Mendoza', 'Nuevo Nivel Norteño', 'Grupo Firme', 
                    'Banda Coloso','La Atrevida Banda Sierra Blanca','Grupo Novedoso',
                    'Super Máquina Musical de Guerrero', 'Sonora Dinamica De Colombia',
                    'La Conga', 'A.B. Quintanilla III Y Los Kumbia Kings', 'Ricardo Muñoz',
                    'Sonador', 'Control', 'El Amigable De Tijuana', 'Tornado','La Hija Del Mariachi',
                    'Ángela Aguilar', 'Los Kiero', 'Banda El RetoÑo', 'Grupo Kual? Dinastia Pedraza'])
artists_lat_reg.extend(['Jenn', 'Tatiana Hazel', 'Danny Daniel','La Montra' ,'Dj Worldwide', 
                        'John Jairo & Dj Ewduarmix'])
artists_not_his.extend(['Hvrbie', 'Franklyn Watts', 'Rhiannon Roze', 'Homestead', 
                        'Chris Jobe', 'Tim Gent', 'Finatticz', 'YUNGHELLBOY', "Jo'el Monroe", 
                        'La Doña','TITUS', 'Jay Pharoah', 'Myles William', 'RØYLS', 'Ashley Clark',
                        'D Nilsz', 'K.P. & Envyi'])
# artists_rock.extend([]) No rock music found in empty genre 

In [19]:
for index in empty_index:
    counter_a = len(set(data.loc[index, 'artists']) & set(artists_bsm))
    counter_b = len(set(data.loc[index, 'artists']) & set(artists_cam))
    counter_c = len(set(data.loc[index, 'artists']) & set(artists_lat_reg))
    counter_d = len(set(data.loc[index, 'artists']) & set(artists_not_his))
    counter_e = len(set(data.loc[index, 'artists']) & set(artists_rock))
    count_max = max([counter_a, counter_b, counter_c, counter_d, counter_e])
    
    if count_max == 0:
        data.loc[index, 'simplified_genre'] = 'Still Empty'
        
    elif count_max == counter_a:  
        data.loc[index, 'simplified_genre'] = 'Bachata Salsa y Merengue'
        
    elif count_max == counter_b:
        data.loc[index, 'simplified_genre'] = 'Cumbia y Musica Mexicana'
        simplified_genres.append('Rock')
        artists_rock.extend(artist_name[i])

        
    elif count_max == counter_c:
        data.loc[index, 'simplified_genre'] = 'Latin Pop and Regg'

       
    elif count_max == counter_d:
        data.loc[index, 'simplified_genre'] = 'Non Hispanic'

    elif count_max == counter_e:
        data.loc[index, 'simplified_genre'] = 'Rock'
        
    

This is the following compostion of my liked music. I will make playlists based on these manually selected Genres

In [22]:
data['simplified_genre'].value_counts(normalize = True)

Cumbia y Musica Mexicana    0.353769
Non Hispanic                0.217085
Latin Pop and Regg          0.214070
Rock                        0.119598
Bachata Salsa y Merengue    0.095477
Name: simplified_genre, dtype: float64

# Playlist Creations

## Playlists (Manual Simplification

## Playlists (Using KMeans) 

## Playlists (Using DBSCAN) 